In [123]:
import numpy as np
import os
import torch
from ase.calculators.vasp import Vasp2
from pathlib import Path
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.lattice import Lattice
from pymatgen.core.structure import Structure
from pymatgen.io.vasp.outputs import Vasprun
from pymatgen.analysis.eos import EOS
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDEntry, PDPlotter
from pymatgen.analysis.pourbaix_diagram import PourbaixEntry, PourbaixDiagram
from pymatgen.entries.computed_entries import ComputedEntry, Composition
from pymatgen.ext.matproj import MPRester
import itertools

from tqdm import tqdm
os.environ["VASP_COMMAND"] = "/private/home/sidgoyal/intel/impi/5.0.2.044/intel64/bin/mpirun -np 64 /private/home/sidgoyal/vasp5.4.4/Linux-x86_64/vasp_parallel"
os.environ["VASP_PP_PATH"] = "/large_experiments/opencatalyst/data/oc22/vasp_pp"

In [124]:
def get_structure(gen, i):
    indexes = torch.cumsum(gen["num_atoms"].squeeze(), dim=0)
    start, end = 0 if i == 0 else indexes[i - 1], indexes[i]
    lengths = gen["lengths"][0][i]
    angles = gen["angles"][0][i]
    atom_types = gen["atom_types"][0][start:end]
    frac_coords = gen["frac_coords"][0][start:end]
    # print(lengths, angles, atom_types.shape, frac_coords.shape)
    return Structure(
        lattice=Lattice.from_parameters(*(lengths.tolist() + angles.tolist())),
        species=atom_types,
        coords=frac_coords,
        coords_are_cartesian=False,
    )

BULK_VASP_FLAGS = {
    'ibrion': 2, 'nsw': 300, 'isif': 3, 'isym':0, 'ediffg': -0.05, 'encut': 500., 'kpts': (10, 10, 10), 
    'prec':'Accurate', 'gga': 'RP', 'pp': 'PBE', 'lwave':False, 'lcharg':False
}

def _clean_up_inputs(atoms, vasp_flags):
    if np.dot(np.cross(atoms.cell[0], atoms.cell[1]), atoms.cell[2]) < 0:
        atoms.set_cell(atoms.cell[[1, 0, 2], :])
    return atoms, vasp_flags

def write_vasp_input_files(atoms, outdir, vasp_flags=None, full_relaxation=True):
    if vasp_flags is None:  # Immutable default
        vasp_flags = BULK_VASP_FLAGS.copy()
    if not full_relaxation:
        vasp_flags['ibrion'] = -1
        vasp_flags['nsw'] = 0
    atoms, vasp_flags = _clean_up_inputs(atoms, vasp_flags)
    calc = Vasp2(directory=outdir, **vasp_flags)
    calc.write_input(atoms)

def create_inputs(gen_file: Path, num: int = None):
    gen = torch.load(gen_file)
    num = num or gen["num_atoms"].shape[1]
    paths = []
    for i in tqdm(range(num), position=1 ,desc=gen_file.name):
        structure = get_structure(gen, i)
        atoms = AseAtomsAdaptor.get_atoms(structure)
        path = gen_file.parent / gen_file.stem / f"{i:05d}"
        write_vasp_input_files(atoms, path)
        paths.append(path)
    paths_file = gen_file.parent / f"{gen_file.stem}_paths.txt"
    with open(paths_file, "w") as f:
        print(*paths, sep="\n", file=f)
    return paths_file

In [126]:
# gen_files = [
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_base/eval_gen.pt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0.1_lr0.001/eval_gen_cond1.pt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0.1_lr0.001/eval_gen_cond6.pt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0.2_lr0.0006/eval_gen_cond1.pt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0.2_lr0.0006/eval_gen_cond6.pt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0_lr0.0008/eval_gen_cond1.pt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0_lr0.0008/eval_gen_cond6.pt",
# ]
# paths = []
# for gen_file in tqdm(gen_files, position=0, desc="gen_files"):
#     paths.append(create_inputs(Path(gen_file), num=500))

In [127]:
# paths = [
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_base/eval_gen_paths.txt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0.1_lr0.001/eval_gen_cond1_paths.txt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0.1_lr0.001/eval_gen_cond6_paths.txt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0.2_lr0.0006/eval_gen_cond1_paths.txt",
#     "exp/cdvae/exp/hydra/singlerun/2023-02-27/mp20_ehull0.2_lr0.0006/eval_gen_cond6_paths.txt",
# ]
# for path in paths:
#     ! echo feje batch-submit-jobs --input-folders-file={path} --project="open_dacc" --calculation-type="vasp" --num-processes=50 --show-progress "&"

# for path in paths:
#     ! echo feje batch-collect-results --input-folders-file={path} --show-progress "&"

# Evaluation

In [128]:
api_key = "dH5V5JF7Hsz5nEeZ5H"
mprester = MPRester(api_key=api_key)

/private/home/anuroops/anaconda3/envs/ocp-20221025/lib/python3.9/site-packages/pymatgen/ext/matproj.py:182: UserWarning:

You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.



In [129]:
def get_ehull(mprester, structure, energy):
    atoms = structure.composition.as_dict().keys()
    url = "/pourbaix_diagram/reference_data/" + "-".join(atoms)
    ion_data = mprester._make_request(url)
    ion_ref_comps = [Composition(d["Reference Solid"]) for d in ion_data]
    ion_ref_elts = list(itertools.chain.from_iterable(i.elements for i in ion_ref_comps))
    ion_ref_entries = mprester.get_entries_in_chemsys(
        list(set([str(e) for e in ion_ref_elts] + ["O", "H"])),
        property_data=["e_above_hull"],
        compatible_only=False)
    phase_diagram = PhaseDiagram(ion_ref_entries)
    pdentry = PDEntry(structure.composition.as_dict(), energy)
    e_hull = phase_diagram.get_e_above_hull(pdentry)
    return e_hull, phase_diagram

In [33]:
path = Path(paths[0].removesuffix("_paths.txt"))
vasp_path = next(path.iterdir())
run = Vasprun(vasp_path / "vasprun.xml")
ehull, phase_diagram = get_ehull(mprester, run.final_structure, run.final_energy)
ehull

/private/home/anuroops/anaconda3/envs/ocp-20221025/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1732: UserWarning: Ignoring unknown variable type SHA256
  warnings.warn(f"Ignoring unknown variable type {key}")
/private/home/anuroops/anaconda3/envs/ocp-20221025/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1732: UserWarning: Ignoring unknown variable type COPYR
  warnings.warn(f"Ignoring unknown variable type {key}")
100%|██████████| 691/691 [00:00<00:00, 1062.18it/s]


0.580379075076527

In [158]:
import pandas as pd
data = pd.read_csv("../data/mp_40atoms_ehull1/test.csv")
row = data.loc[38]
cif, energy, ehull_gt = row.cif, row.formation_energy_per_atom, row.e_above_hull
structure = Structure.from_str(cif, fmt="cif")
# energy *= len(structure)
# ehull, _ = get_ehull(mprester, structure, energy)
structure.composition.reduced_composition, energy, ehull_gt

(Comp: Al1 O2, -2.421029230833333, 0.5256319512121186)

In [153]:
elements = list(elt.name for elt in structure.composition.elements)
print(elements)
entries = mprester.get_entries_in_chemsys(elements)
ce = ComputedEntry(structure.composition, energy)
pd = PhaseDiagram(entries + [ce])
pd

['Al', 'O']


Al-O phase diagram
3 stable phases: 
Al2O3, O2, Al

In [154]:
plotter = PDPlotter(pd)
plotter.show()

In [151]:
# ehull2 = pd.get_e_above_hull(ce) / len(structure)
# ehull2
# pd.get_e_above_hull(ce)
# pd.get_decomp_and_e_above_hull(ce)[1]

# decomp, hull_energy = pd.get_decomp_and_hull_energy_per_atom(ce.composition)
# decomp, hull_energy
# ce.energy_per_atom, hull_energy

decomp = pd.get_decomposition(ce.composition)
# sum(e.energy_per_atom * n for e, n in decomp.items())
print([(e.energy_per_atom, n) for e, n in decomp.items()])

[(-5.7798218, 0.5), (-4.09920667, 0.5)]


['Ce', 'Mg', 'Al']

In [50]:
atoms = structure.composition.as_dict().keys()
url = "/pourbaix_diagram/reference_data/" + "-".join(atoms)
ion_data = mprester._make_request(url)

In [ ]:
ion_ref_comps = [Composition(d["Reference Solid"]) for d in ion_data]
ion_ref_elts = list(itertools.chain.from_iterable(i.elements for i in ion_ref_comps))
ion_ref_comps, ion_ref_elts

In [56]:
ion_ref_entries = mprester.get_entries_in_chemsys(
    list(set([str(e) for e in ion_ref_elts] + ["O", "H"])),
    property_data=["e_above_hull"],
    compatible_only=False)
ion_ref_entries

[mp-1239196 ComputedEntry - Al4          (Al)
 Energy (Uncorrected)     = -13.7714  eV (-3.4428  eV/atom)
 Correction               = 0.0000    eV (0.0000   eV/atom)
 Energy (Final)           = -13.7714  eV (-3.4428  eV/atom)
 Energy Adjustments:
   None
 Parameters:
   run_type               = GGA
   is_hubbard             = False
   pseudo_potential       = {'functional': 'PBE', 'labels': ['Al'], 'pot_type': 'paw'}
   hubbards               = {}
   potcar_symbols         = ['PBE Al']
   oxide_type             = None
 Data:
   oxide_type             = None
   e_above_hull           = 0.30273657750000016,
 mp-1183144 ComputedEntry - Al4          (Al)
 Energy (Uncorrected)     = -14.9365  eV (-3.7341  eV/atom)
 Correction               = 0.0000    eV (0.0000   eV/atom)
 Energy (Final)           = -14.9365  eV (-3.7341  eV/atom)
 Energy Adjustments:
   None
 Parameters:
   run_type               = GGA
   is_hubbard             = False
   pseudo_potential       = {'functional': 'PBE', 'la

In [ ]:
phase_diagram = PhaseDiagram(ion_ref_entries)
pdentry = PDEntry(structure.composition.as_dict(), energy)
e_hull = phase_diagram.get_e_above_hull(pdentry)